In [1]:
import numpy as np

In [2]:
def phase_prog(original_program, correction_angle, n_cycles):
    glob_phase = 0
    new_program = []
    for i in range(n_cycles):
        for j in range(len(original_program)):
            glob_phase += correction_angle;
            new_program.append( (original_program[j]+glob_phase) % 360)
            
    return new_program

In [3]:
ken16 = [0,90,90,0,0,90,90,0,180,270,270,180,180,270,270,180]

In [4]:
temp = phase_prog(ken16,2,16)
"".join(str(phase)+" " for phase in temp)

'2 94 96 8 10 102 104 16 198 290 292 204 206 298 300 212 34 126 128 40 42 134 136 48 230 322 324 236 238 330 332 244 66 158 160 72 74 166 168 80 262 354 356 268 270 2 4 276 98 190 192 104 106 198 200 112 294 26 28 300 302 34 36 308 130 222 224 136 138 230 232 144 326 58 60 332 334 66 68 340 162 254 256 168 170 262 264 176 358 90 92 4 6 98 100 12 194 286 288 200 202 294 296 208 30 122 124 36 38 130 132 44 226 318 320 232 234 326 328 240 62 154 156 68 70 162 164 76 258 350 352 264 266 358 0 272 94 186 188 100 102 194 196 108 290 22 24 296 298 30 32 304 126 218 220 132 134 226 228 140 322 54 56 328 330 62 64 336 158 250 252 164 166 258 260 172 354 86 88 0 2 94 96 8 190 282 284 196 198 290 292 204 26 118 120 32 34 126 128 40 222 314 316 228 230 322 324 236 58 150 152 64 66 158 160 72 254 346 348 260 262 354 356 268 90 182 184 96 98 190 192 104 286 18 20 292 294 26 28 300 122 214 216 128 130 222 224 136 318 50 52 324 326 58 60 332 '

In [5]:
pine8 = [0,0,180,180,180,180,0,0]

In [6]:
def z_prog_trotter(program, theta, cyc_max):
    r"""
    Computes the phase program for a phase program primitive, ``program``,
    and a trotterized virtual Z gate of angle ``theta``, for up to 
    ``cyc_max`` Hamiltonian engineering cycles. Sequence is
    
    U = ( Z(theta) * U(program) * Z(theta) )^k
      = Z(theta) * U(program) * Z(2*theta) * U(program) * ... * Z(2*theta) * U(program) * Z(theta)
    
    """
    glob_phase = theta
    new_program = []
    for _ in range(cyc_max):
        for phase in program:
            new_program.append((phase+glob_phase)%360)
        glob_phase += 2*theta
    glob_phase += theta
    return new_program

In [7]:
temp = z_prog_trotter(pine8,150,16)
"".join(str(phase)+" " for phase in temp)

'150 150 330 330 330 330 150 150 90 90 270 270 270 270 90 90 30 30 210 210 210 210 30 30 330 330 150 150 150 150 330 330 270 270 90 90 90 90 270 270 210 210 30 30 30 30 210 210 150 150 330 330 330 330 150 150 90 90 270 270 270 270 90 90 30 30 210 210 210 210 30 30 330 330 150 150 150 150 330 330 270 270 90 90 90 90 270 270 210 210 30 30 30 30 210 210 150 150 330 330 330 330 150 150 90 90 270 270 270 270 90 90 30 30 210 210 210 210 30 30 330 330 150 150 150 150 330 330 '

In [8]:
def z_prog_trotter_alt(program, theta, cyc_max):
    r"""
    Computes the phase program for a phase program primitive, ``program``,
    and a trotterized virtual Z gate of angle ``theta``, for up to 
    ``cyc_max`` Hamiltonian engineering cycles. Sequence is
    
    U = ( Z(theta) * U(program) )^k
      = Z(theta) * U(program) * Z(theta) * U(program) * ... * Z(theta) * U(program)
    
    """
    glob_phase = 0
    new_program = []
    for _ in range(cyc_max):
        for phase in program:
            new_program.append((phase+glob_phase)%360)
        glob_phase += theta
    return new_program, glob_phase

In [9]:
temp, final = z_prog_trotter_alt(pine8,50,16)
"".join(str(phase)+" " for phase in temp)

'0 0 180 180 180 180 0 0 50 50 230 230 230 230 50 50 100 100 280 280 280 280 100 100 150 150 330 330 330 330 150 150 200 200 20 20 20 20 200 200 250 250 70 70 70 70 250 250 300 300 120 120 120 120 300 300 350 350 170 170 170 170 350 350 40 40 220 220 220 220 40 40 90 90 270 270 270 270 90 90 140 140 320 320 320 320 140 140 190 190 10 10 10 10 190 190 240 240 60 60 60 60 240 240 290 290 110 110 110 110 290 290 340 340 160 160 160 160 340 340 30 30 210 210 210 210 30 30 '

In [10]:
def z_prog_trotter_alt_xmeas(program, theta, cyc_max):
    r"""
    Computes the phase program for a phase program primitive, ``program``,
    and a trotterized virtual Z gate of angle ``theta``, for up to 
    ``cyc_max`` Hamiltonian engineering cycles. Sequence is
    
    U = Y(3*pi/2) * ( Z(theta) * U(program) )^k
      = Y(3*pi/2) * Z(theta) * U(program) * Z(theta) * U(program) * ... * Z(theta) * U(program)
    
    """
    glob_phase = 0
    mp = 270
    new_program = []
    meas_program = [mp, (mp+180) % 360, (mp+180)%360, mp]
    for _ in range(cyc_max):
        for phase in program:
            new_program.append((phase+glob_phase) % 360)
        glob_phase -= theta
        for ph in [mp, mp+180, mp+180, mp]:
            meas_program.append((ph+glob_phase) % 360)
    return new_program, meas_program

In [67]:
temp, meas = z_prog_trotter_alt_xmeas(pine8,12,16)
" ".join(str(phase) for phase in temp)

'0 0 180 180 180 180 0 0 348 348 168 168 168 168 348 348 336 336 156 156 156 156 336 336 324 324 144 144 144 144 324 324 312 312 132 132 132 132 312 312 300 300 120 120 120 120 300 300 288 288 108 108 108 108 288 288 276 276 96 96 96 96 276 276 264 264 84 84 84 84 264 264 252 252 72 72 72 72 252 252 240 240 60 60 60 60 240 240 228 228 48 48 48 48 228 228 216 216 36 36 36 36 216 216 204 204 24 24 24 24 204 204 192 192 12 12 12 12 192 192 180 180 0 0 0 0 180 180'

In [68]:
" ".join(str(phase) for phase in meas)

'270 90 90 270 258 78 78 258 246 66 66 246 234 54 54 234 222 42 42 222 210 30 30 210 198 18 18 198 186 6 6 186 174 354 354 174 162 342 342 162 150 330 330 150 138 318 318 138 126 306 306 126 114 294 294 114 102 282 282 102 90 270 270 90 78 258 258 78'